In [1]:
! pip install pyspark

     |████████████████████████████████| 215.6MB 55.6MB/s 
     |████████████████████████████████| 204kB 41.5MB/s 
  Created wheel for pyspark: filename=pyspark-2.4.3-py2.py3-none-any.whl size=215965824 sha256=32c2df50952f0315706a51168fcd9393066a793e496bbc53227f16e0500aa10c
  Stored in directory: /tmp/.cache/pip/wheels/8d/20/f0/b30e2024226dc112e256930dd2cd4f06d00ab053c86278dcf3
Successfully built pyspark


In [2]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/titanic/gender_submission.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/train.csv


In [3]:
from pyspark.sql import SparkSession
from pyspark.sql import DataFrame 
from pyspark.sql import functions as F
from pyspark.ml.feature import Imputer, StringIndexer, OneHotEncoderEstimator, VectorAssembler, PCA
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier

spark = SparkSession.builder.getOrCreate()
spark

In [4]:
sdf_train = spark.read.csv('/kaggle/input/titanic/train.csv', inferSchema = "true", header = "true")
sdf_train.toPandas().T

,0,1,2,3,4,5,6,7,8,9,...,881,882,883,884,885,886,887,888,889,890
PassengerId,1,2,3,4,5,6,7,8,9,10,...,882,883,884,885,886,887,888,889,890,891
Survived,0,1,1,1,0,0,0,0,1,1,...,0,0,0,0,0,0,1,0,1,0
Pclass,3,1,3,1,3,3,1,3,3,2,...,3,3,2,3,3,2,1,3,1,3
Name,"Braund, Mr. Owen Harris","Cumings, Mrs. John Bradley (Florence Briggs Th...","Heikkinen, Miss. Laina","Futrelle, Mrs. Jacques Heath (Lily May Peel)","Allen, Mr. William Henry","Moran, Mr. James","McCarthy, Mr. Timothy J","Palsson, Master. Gosta Leonard","Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)","Nasser, Mrs. Nicholas (Adele Achem)",...,"Markun, Mr. Johann","Dahlberg, Miss. Gerda Ulrika","Banfield, Mr. Frederick James","Sutehall, Mr. Henry Jr","Rice, Mrs. William (Margaret Norton)","Montvila, Rev. Juozas","Graham, Miss. Margaret Edith","""Johnston, Miss. Catherine Helen """"Carrie""""""","Behr, Mr. Karl Howell","Dooley, Mr. Patrick"
Sex,male,female,female,female,male,male,male,male,female,female,...,male,female,male,male,female,male,female,female,male,male
Age,22,38,26,35,35,NaN,54,2,27,14,...,33,22,28,25,39,27,19,NaN,26,32
SibSp,1,1,0,1,0,0,0,3,0,1,...,0,0,0,0,0,0,0,1,0,0
Parch,0,0,0,0,0,0,0,1,2,0,...,0,0,0,0,5,0,0,2,0,0
Ticket,A/5 21171,PC 17599,STON/O2. 3101282,113803,373450,330877,17463,349909,347742,237736,...,349257,7552,C.A./SOTON 34068,SOTON/OQ 392076,382652,211536,112053,W./C. 6607,111369,370376
Fare,7.25,71.2833,7.925,53.1,8.05,8.4583,51.8625,21.075,11.1333,30.0708,...,7.8958,10.5167,10.5,7.05,29.125,13,30,23.45,30,7.75


In [5]:
sdf_test = spark.read.csv("/kaggle/input/titanic/test.csv", inferSchema = "true", header = "true")
sdf_test.toPandas().T

,0,1,2,3,4,5,6,7,8,9,...,408,409,410,411,412,413,414,415,416,417
PassengerId,892,893,894,895,896,897,898,899,900,901,...,1300,1301,1302,1303,1304,1305,1306,1307,1308,1309
Pclass,3,3,2,3,3,3,3,2,3,3,...,3,3,3,1,3,3,1,3,3,3
Name,"Kelly, Mr. James","Wilkes, Mrs. James (Ellen Needs)","Myles, Mr. Thomas Francis","Wirz, Mr. Albert","Hirvonen, Mrs. Alexander (Helga E Lindqvist)","Svensson, Mr. Johan Cervin","Connolly, Miss. Kate","Caldwell, Mr. Albert Francis","Abrahim, Mrs. Joseph (Sophie Halaut Easu)","Davies, Mr. John Samuel",...,"Riordan, Miss. Johanna Hannah""""""""","Peacock, Miss. Treasteall","Naughton, Miss. Hannah","Minahan, Mrs. William Edward (Lillian E Thorpe)","Henriksson, Miss. Jenny Lovisa","Spector, Mr. Woolf","Oliva y Ocana, Dona. Fermina","Saether, Mr. Simon Sivertsen","Ware, Mr. Frederick","Peter, Master. Michael J"
Sex,male,female,male,male,female,male,female,male,female,male,...,female,female,female,female,female,male,female,male,male,male
Age,34.5,47,62,27,22,14,30,26,18,21,...,NaN,3,NaN,37,28,NaN,39,38.5,NaN,NaN
SibSp,0,1,0,0,1,0,0,1,0,2,...,0,1,0,1,0,0,0,0,0,1
Parch,0,0,0,0,1,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,1
Ticket,330911,363272,240276,315154,3101298,7538,330972,248738,2657,A/4 48871,...,334915,SOTON/O.Q. 3101315,365237,19928,347086,A.5. 3236,PC 17758,SOTON/O.Q. 3101262,359309,2668
Fare,7.8292,7,9.6875,8.6625,12.2875,9.225,7.6292,29,7.2292,24.15,...,7.7208,13.775,7.75,90,7.775,8.05,108.9,7.25,8.05,22.3583
Cabin,None,None,None,None,None,None,None,None,None,None,...,None,None,None,C78,None,None,C105,None,None,None


In [6]:
numeric_cols = ['PassengerId','Survived', 'Pclass','Age', 'SibSp','Parch','Ticket','Fare'] 
numeric_features = ['PassengerId','Pclass','Age', 'SibSp','Parch','Fare'] 
numeric_cols_test = ['PassengerId', 'Pclass','Age', 'SibSp','Parch','Ticket','Fare', 'Sex'] 
string_features = [ 'Embarked', 'Sex']

In [7]:
def _make_numeric(sdf: DataFrame, str_col_name: str) -> DataFrame:
    sdf = sdf.withColumn(str_col_name, sdf[str_col_name].cast('double'))
    return sdf
sdf_train = _make_numeric(sdf_train,'Ticket')
sdf_test = _make_numeric(sdf_test,'Ticket')
# sdf_train.toPandas().T

In [8]:
def _impute_median(sdf: DataFrame,lst_columns: list) -> DataFrame:
    imputer = Imputer(inputCols = lst_columns, outputCols = lst_columns)
    sdf = imputer.fit(sdf).transform(sdf)
    return sdf
lst_cols_to_impute = ['Fare','Age']
sdf_train_imputed = _impute_median(sdf_train,lst_cols_to_impute)
sdf_test_imputed = _impute_median(sdf_test,lst_cols_to_impute)

In [9]:
# sdf_train_imputed.toPandas().T

In [10]:
# def _str_indexer():
_stages = []
string_indexer =  [StringIndexer(inputCol = column , \
                                 outputCol = column + '_indx', handleInvalid = 'skip') for column in string_features]
_stages += string_indexer

one_hot_encoder = [OneHotEncoderEstimator(inputCols = [column + '_indx' for column in string_features ], \
                                          outputCols =  [column + '_encoded' for column in string_features ])]
_stages += one_hot_encoder

assemblerInput =  [f  for f in numeric_features]  
assemblerInput += [f + "_encoded" for f in string_features]
vector_assembler = VectorAssembler(inputCols = assemblerInput, \
                                   outputCol = 'vect_features')
_stages += [vector_assembler]

pca = PCA(inputCol = 'vect_features', outputCol = 'pca_features', k = 5)
_stages += [pca]

rf = RandomForestClassifier(labelCol = 'Survived', featuresCol = 'pca_features', numTrees = 100, maxDepth = 4, maxBins = 1000)
_stages += [rf]

In [11]:
pipeline = Pipeline(stages = _stages)

In [12]:
model = pipeline.fit(sdf_train_imputed)

In [13]:
sdf_predict = model.transform(sdf_test_imputed)

In [14]:
sdf_predict.toPandas().T

,0,1,2,3,4,5,6,7,8,9,...,408,409,410,411,412,413,414,415,416,417
PassengerId,892,893,894,895,896,897,898,899,900,901,...,1300,1301,1302,1303,1304,1305,1306,1307,1308,1309
Pclass,3,3,2,3,3,3,3,2,3,3,...,3,3,3,1,3,3,1,3,3,3
Name,"Kelly, Mr. James","Wilkes, Mrs. James (Ellen Needs)","Myles, Mr. Thomas Francis","Wirz, Mr. Albert","Hirvonen, Mrs. Alexander (Helga E Lindqvist)","Svensson, Mr. Johan Cervin","Connolly, Miss. Kate","Caldwell, Mr. Albert Francis","Abrahim, Mrs. Joseph (Sophie Halaut Easu)","Davies, Mr. John Samuel",...,"Riordan, Miss. Johanna Hannah""""""""","Peacock, Miss. Treasteall","Naughton, Miss. Hannah","Minahan, Mrs. William Edward (Lillian E Thorpe)","Henriksson, Miss. Jenny Lovisa","Spector, Mr. Woolf","Oliva y Ocana, Dona. Fermina","Saether, Mr. Simon Sivertsen","Ware, Mr. Frederick","Peter, Master. Michael J"
Sex,male,female,male,male,female,male,female,male,female,male,...,female,female,female,female,female,male,female,male,male,male
Age,34.5,47,62,27,22,14,30,26,18,21,...,30.2726,3,30.2726,37,28,30.2726,39,38.5,30.2726,30.2726
SibSp,0,1,0,0,1,0,0,1,0,2,...,0,1,0,1,0,0,0,0,0,1
Parch,0,0,0,0,1,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,1
Ticket,330911,363272,240276,315154,3.1013e+06,7538,330972,248738,2657,NaN,...,334915,NaN,365237,19928,347086,NaN,NaN,NaN,359309,2668
Fare,7.8292,7,9.6875,8.6625,12.2875,9.225,7.6292,29,7.2292,24.15,...,7.7208,13.775,7.75,90,7.775,8.05,108.9,7.25,8.05,22.3583
Cabin,None,None,None,None,None,None,None,None,None,None,...,None,None,None,C78,None,None,C105,None,None,None


In [15]:
sdf_submission = sdf_predict.select('PassengerId','prediction')\
                            .withColumn('Survived',sdf_predict['prediction'].cast('integer'))\
                            .select('PassengerId','Survived')
sdf_submission.toPandas().T

,0,1,2,3,4,5,6,7,8,9,...,408,409,410,411,412,413,414,415,416,417
PassengerId,892,893,894,895,896,897,898,899,900,901,...,1300,1301,1302,1303,1304,1305,1306,1307,1308,1309
Survived,0,0,0,0,1,0,0,1,0,0,...,0,1,0,1,0,0,1,0,0,1


In [16]:
sdf_submission.coalesce(1).write.csv("submission",mode="overwrite",header=True)

In [17]:
print(os.listdir('submission'))


['.part-00000-2b75c9c8-f514-4415-b926-2ca267fe549f-c000.csv.crc', '_SUCCESS', '._SUCCESS.crc', 'part-00000-2b75c9c8-f514-4415-b926-2ca267fe549f-c000.csv']


<a href='submission/part-00000-f0fdb22b-1f16-4bb9-8345-f434201a00ae-c000.csv'>download result</a>

TODO:  
https://spark.apache.org/docs/latest/ml-features.html#vectorindexer  
https://spark.apache.org/docs/latest/ml-features.html#bucketizer  

In [18]:
Todo:

SyntaxError: invalid syntax (<ipython-input-18-a3ecb612cd34>, line 1)